In [1]:
from random import randint, randrange
import string
from time import time
from tkinter.tix import Tree
from turtle import width
import pandas as pd
from plotly.subplots import make_subplots
import numpy as np
import matplotlib.pyplot as plt
from scipy import rand
from tqdm import tqdm
import mplfinance as mplf
import plotly.graph_objects as go

In [2]:
# In this script we add noise to candles. There are different ways of
# adding noise to candles and all the different methods are in the Noise
# class.
def getCandles(fileLocation : string) -> pd.DataFrame():
    # This fuunction reads a dataframe from a csv file
    return pd.read_csv(fileLocation)

def plotDataset(df:pd.DataFrame, method = "mplfinance") -> None:
    """
    The function to plot candlestick data.
    
    Arguments
    ---------
    df: pd.DataFrame: The dataframe containing open, high, close and low of candles in each column. 
        The dataframe has to contain a "date" column representing the open time of each cnadle. the
        dataframe can contain "close_time" and "volume" column but it is not necessary to provide 
        them.
    method: string: The method to plot the data. (Choose between "mplfinance" or "plotly")

    Returns
    -------
    None

    """
    
    if method == "mplfinance":
        import mplfinance as mplf
        
        # Setting the date column as index
        df.index = pd.DatetimeIndex(df['date'])

        mplf.plot(
            df,
            style="charles",
            type="candle",
            volume=False , # if "volume" not in df.columns else True, 
            title={"title": "Candlestick data"},
            tight_layout=True,
        )

    elif method == "plotly":
        import plotly.graph_objects as go   

        noisedChart = go.Candlestick(
                x=df["date"],
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'],
                name = "Candlestick data",
                # increasing={'line': {'color': 'blue'}},
                # decreasing={'line': {'color': 'purple'}},
                )

        layout = go.Layout(
            title = 'Overview',
            xaxis = go.XAxis(
                title = 'Date',
                rangeslider_visible = False,
                showticklabels=False),
            yaxis = go.YAxis(
                title = 'Price',
                fixedrange = False
            )
        )

        fig = go.Figure(data=[noisedChart], layout = layout)

        fig.show()

    else:
        print("Error, Method only mplfinance or plotly are allowed")

def saveCandles(df:pd.DataFrame, location:string, saveImage = False, method = "mplfinance") -> None:
    """
    The function to save the candlestick data to a pickle file.

    Arguments
    ---------
    df: pd.DataFrame: The dataframe containing open, high, close and low of candles in each column. 
        The dataframe has to contain a "date" column representing the open time of each cnadle. the
        dataframe can contain "close_time" and "volume" column but it is not necessary to provide 
        them.    
    location: string: The location of the file to save the data. It should contain file's name and 
    also it's extention.
    saveImage: boolean: If true the candlestick plot is saved as an image.
    method: string: The method to plot the data. (Choose between "mplfinance" or "plotly")

    Returns
    -------
    None

    Note
    ----
    * To use plotly as a method of saving images, you need to have kaleido package installed.

    """

    if saveImage:
        if method == "mplfinance":    
            import mplfinance as mplf
            
            # Setting the date column as index
            df.index = pd.DatetimeIndex(df['date'])

            mplf.plot(
                df,
                style="charles",
                type="candle",
                volume=False if "volume" not in df.columns else True, 
                title={"title": "Candlestick data"},
                tight_layout=True,
                savefig = location.replace(".pkl",".png")
            )
        elif method == "plotly":
            import plotly.graph_objects as go   

            noisedChart = go.Candlestick(
                    x=df["date"],
                    open=df['open'],
                    high=df['high'],
                    low=df['low'],
                    close=df['close'],
                    name = "Candlestick data",
                    # increasing={'line': {'color': 'blue'}},
                    # decreasing={'line': {'color': 'purple'}},
                    )

            layout = go.Layout( margin=dict(l=20, r=20, t=20, b=20))
            fig = go.Figure(data=[noisedChart], layout=layout)
            fig.update_xaxes(rangeslider_visible=False,  title = location[location.rfind("/")+1:location.rfind(".")])
            fig.write_image(location.replace(".pkl",".png"), width=800, height=400)

    df.to_pickle(location)




In [3]:
class Noise():
    """
    The master class that contains the different methods of adding noise
    to dataset.
    """

    def __init__(self) -> None:
        """
        Initialize the class
        """
        pass

    def GaussianSeries(self, dataset: pd.Series, scale) -> pd.Series(dtype = "float64"):
        """
        Gets the gaussian noise for dataframe series. The mean value for calculation is
        the value of each row. The calculation formula of noise for a mean(u) and standard
        deviation (sigma) is stated below

        Noise(z) = exp(-(z-u)^2/2/sigma^2)/sigma/sqrt(2*pi)
        
        Arguments
        ---------
            dataset: a pandas series to add noise to
            scale: The standard deviation of the noise that will be added (In percentage units)

        
        Returns
        -------
            A pandas dataframe containing past data + Noise
        """
        scalePercent = scale / 100

        dataset = dataset.apply(lambda x: x + np.random.normal(loc = 0, scale = x * scalePercent, size = 1).item())

        return dataset

    def GaussianCnadles(self,
        time:pd.Series,
        open:pd.Series, 
        high:pd.Series, 
        close:pd.Series, 
        low:pd.Series, 
        volume:pd.Series, 
        close_time:pd.Series, 
        scale, 
        method: int
        ) -> pd.DataFrame(dtype="float64"):
        """
        Method that will add gaussian noise to a candlesstick dataset. There are two methods supported for now

        method1: The noise will be added to all the parameters of each candle. this will cause the candles to be
        non-continuous, i.e. The close of previous candle will not be equal to open of the current candle. (provided
        that the input candles are continous)

        method2: Adding the noise to high, low and the body of the candle. In this method open price of the first
        candle in dataset will be the same but the open of other candles will be cahnged with respect to the noise of 
        the candle's body (but the candles will stay continous).
        
        Arguments
        ---------
            open, high low, close: pd.Series: all pandas dataframes (or series)
            scale: [float]: The standard deviation of the noise that will be added respectively
            to body, upper wick and lower wick of the candles. if only one variable is passed 
            inside the array, it will be assigend to all the parts of the candle. (The scale has
            to be in percentage units)
        
        Returns
        -------
            A pandas dataframe with columns = ["time","open","high","low","close"]
        """
        _open, _high, _close, _low = None, None, None, None
        shape = time.shape[0]
        
        # Assigning the scales of noise in each part of candle
        if   len(scale) == 1:
            scale_body, scale_high, scale_low = 1 + scale[0] / 100, 1 + scale[0] / 100, 1 + scale[0] / 100
        elif len(scale) == 2:
            scale_body, scale_high, scale_low = 1 + scale[0] / 100, 1 + scale[1] / 100, 1 + scale[1] / 100
        elif len(scale) == 3:
            scale_body, scale_high, scale_low = 1 + scale[0] / 100, 1 + scale[1] / 100, 1 + scale[2] / 100

        # Convert series to dataframe
        frame = {"time":time,"open":open,"high":high,"low":low,"close":close}
        df = pd.DataFrame(frame)

        # We added this method to see if the prices contain negative value (which are not acceptable)
        # Only after the end of data generation we can know this, so we use a while loop 
        _continue = True

        while _continue:
            if method == 1:
                # The logic: In thsi method we do not care about not generating gaps. so we add noise
                # to the body of each candle but we keep the open price the same. this will avoid the 
                # noise to be accumulated and chart looks very much like the original (Without noise)
                
                df["Date"] = time
                df["open"] = open
                df["close"] = close
                df["high"] = high
                df["low"] = low
                df["volume"] = volume

                shape = time.shape[0]
                pert_body = np.random.normal(loc = 0, scale = 1, size = shape)
                pert_body /= max(pert_body)

                pert_upperWick = np.random.normal(loc = 0, scale = 1, size = shape)
                pert_upperWick /= max(pert_upperWick)

                pert_lowerWick = np.random.normal(loc = 0, scale = 1, size = shape)
                pert_lowerWick /= max(pert_lowerWick)

                temp = pd.DataFrame()
                temp["u1"] = df["high"] - df["close"]
                temp["u2"] = df["high"] - df["open"]
                temp["l1"] = df["close"] - df["low"]
                temp["l2"] = df["open"] - df["low"]

                _upperWickLen = temp[["u1","u2"]].max(axis=1)
                _lowerWickLen = temp[["u1","u2"]].max(axis=1)
                _bodyLen = close - open

                _body = _bodyLen * (1 + pert_body) * scale_body
                _high = _upperWickLen * (1 + pert_upperWick) * scale_high
                _low  = _lowerWickLen * (1 + pert_lowerWick) * scale_low
                _volume = volume * abs((_high - _low)/(high - low))

                df["_open"] = open
                df["_close"] = open + _body
                df["_high"] =  df[["_open","_close"]].max(axis=1) +_high
                df["_low"] =  df[["_open","_close"]].min(axis=1) +_low
                df["_volume"] = _volume

                frame = {
                    "date":time,
                    "open":df["_open"],
                    "high":df["_high"],
                    "low":df["_low"],
                    "close":df["_close"],
                    "volume":df["_volume"],
                    "close_time":close_time
                }

                returnDF = pd.DataFrame(frame)

                # Generate the data untill there are no negative values
                if not (returnDF.iloc[:,1:4] < 0).any().any():
                    _continue = False

            elif method == 2:
                # The logic: to keep the candles continouse and avoid making gamps, we use cumsum 
                # of the body + noise adn assign the value to the candle's close. and then we assign
                # the close of the current candle to open of the next candle
                
                df["Date"] = time
                df["open"] = open
                df["close"] = close
                df["high"] = high
                df["low"] = low
                df["volume"] = volume

                shape = time.shape[0]
                pert_body = np.random.normal(loc = 0, scale = 1, size = shape)
                pert_body /= max(pert_body)

                pert_upperWick = np.random.normal(loc = 0, scale = 1, size = shape)
                pert_upperWick /= max(pert_upperWick)

                pert_lowerWick = np.random.normal(loc = 0, scale = 1, size = shape)
                pert_lowerWick /= max(pert_lowerWick)

                temp = pd.DataFrame()
                temp["u1"] = df["high"] - df["close"]
                temp["u2"] = df["high"] - df["open"]
                temp["l1"] = df["close"] - df["low"]
                temp["l2"] = df["open"] - df["low"]

                _upperWickLen = temp[["u1","u2"]].max(axis=1)
                _lowerWickLen = temp[["u1","u2"]].max(axis=1)
                _bodyLen = close - open

                _body = _bodyLen * (1 + pert_body) * scale_body
                _high = _upperWickLen * (1 + pert_upperWick) * scale_high
                _low  = _lowerWickLen * (1 + pert_lowerWick) * scale_low
                _volume = volume * abs((_high - _low)/(high - low))

                _close = np.cumsum(_body) + close.iloc[0]
                df["_close"] = _close


                df["_open"] = df["_close"].shift(1)
                _k = df.columns.get_loc("_open")
                k = df.columns.get_loc("open")
                df.iloc[0,_k] = df.iloc[0,k]

                df["_high"] = df[["_open","_close"]].max(axis=1) + _high
                df["_low"]  = df[["_open","_close"]].min(axis=1) - _low

                df["_volume"] = _volume


                frame = {
                    "date":time,
                    "open":df["_open"],
                    "high":df["_high"],
                    "low":df["_low"],
                    "close":df["_close"],
                    "volume":df["_volume"],
                    "close_time":close_time
                }

                returnDF = pd.DataFrame(frame)

                
                # Generate the data untill there are no negative values
                if not (returnDF.iloc[:,1:4] < 0).any().any():
                    _continue = False

            else:
                print("Error, wrong method number provided!")
                
                _continue = False
        
        return returnDF

In [21]:
from matplotlib import scale


class GenerateCandles(Noise):
    """
    This class is for making random stock candlestick charts with different methods
    """

    def __init__(self) -> None:
        
        pass

    def BrownianMotion(self, candlesCount:int, volatility:float, wicks:list, maxMargin, mean = 0, startDate = "2022-01-01", frequency = "5min") -> pd.DataFrame:
        """
        Generates a brownian motion (Also know ans wiener process) with the given parameters.
        Note that the brownian motion is used to generate random stock prices as a time series 
        representable in a line chart. But in some startegies we need candlestick data (open, 
        high, low, close) to be able to run the backtest. We have chosen the random variables 
        generated by wiener process as the closing price of each candle. Also the oppening of
        the next candle is the closing of the previous candle. The only challange is to generate
        the high and low of the candle. we have used a normal distribution to generate theese 
        values but the maximum limit for the shadows will be a percentage of the cnalde body.
        This value will be a parameter of the function.
        
        Updates:
        --------
        1. Some charting applications require indexes to be date, so to make the data more suitable 
        for these applications (i.e. mplfinance) we add a date column as well to the outputted 
        dataframe. The starting date of the data is set to 2022-01-01 as a default but it can be 
        changed by the user.

        2. The candlestick data has to be positive, but the wiener process generates random numbers
        and these random numbers can be negative. So we have to make sure that the candlestick
        dataset is positive. To do this, we generate a random number between 1 and "maxMargin"
        and shift the chart (If it has negative prices) to the generated number.



        Parameters
        ----------
        candlesCount: int: number of candles to generate
        mean: float: mean of the brownian motion, The default is 0
        volatility: float: variance of the brownian motion
        wicks: list[lower, upper]: the length of the upper and lower wicks as a percentage of the
            candle body The values has to be in percentage(e.g. 50 for 50%)
        startDate: str: starting date of the data (optional)
        frequency: str: frequency of the data (optional)
        
        Updates:
        --------
        1. In previouse versions of the function, the body's data was generated by a normal destribution
        around zer. While this method provided us with good results but In this way, the method was 
        incapable of including fat tails (The very excentric candles that we see in markets) in the dataset.
        To counter this issue we first take a uniform distribution in desired price range, This is our 
        volatility. In the second step, we take the generated numbers, And find a normal distribution 
        around it (Again, with our desired volatility). This is our body.


        To do:
        ------
        * Adding the volume for each candle (The volume will be proportional to the candle's body).
        Currently the volume is set to 0.

        Returns
        -------
        df: pandas.DataFrame: The generated candles containing the date, open, high, low, close, 
        close_time and volume. (date is the oppening time of the candle)
        """

        # Because of shifting, we lose the last candle, so we add one to the candlesCount
        candlesCount += 1
        wicks = [n/100 for n in wicks]

        # The dataframe to be returned
        df = pd.DataFrame()
        
        V_Volatility = np.random.uniform(low = -60, high = 60, size = candlesCount) # The volatility of the noise
        print(len(V_Volatility))
        random = np.zeros([candlesCount, 1])
        j = 0
        for num in V_Volatility:
            random[j] =  [np.random.normal(scale = 60, loc = num, size = 1)]
            j += 1


        # random = np.random.normal(loc = mean, scale = 1 * volatility, size = candlesCount)

        df["date"] = pd.date_range(start = startDate, periods = candlesCount, freq = "5min")
        df["open"] = random.cumsum()
        df["close"] = df["open"].shift(-1)

        _body = df["close"] - df["open"]
        
        # The amount of nose to for generating the high and low wicks
        rand1 = np.random.normal(loc = 0, scale = 1 , size = candlesCount); rand1 = rand1/abs(rand1).max()
        rand2 = np.random.normal(loc = 0, scale = 1 , size = candlesCount); rand2 = rand2/abs(rand2).max()
        
        up = df[["open","close"]].max(axis=1)
        down = df[["open","close"]].min(axis=1)

        df["low"] = down - abs(_body) * abs(1 + wicks[0]) * abs(rand2)
        df["high"] = up + abs(_body) * abs(1 + wicks[1]) * abs(rand1)
        df["volume"] = 0
        df["close_time"] = df["date"].shift(-1)

        # Shifting the data to make negative prices positive
        lowestNum = df["low"].min()

        if lowestNum < 0:
            shift = randint(1,maxMargin)

            df["open"] += shift -lowestNum
            df["close"] += shift -lowestNum
            df["low"] += shift -lowestNum
            df["high"] += shift -lowestNum

        return df.iloc[:-2,:]

In [24]:

cls = GenerateCandles()

candles = cls.BrownianMotion(
    candlesCount = 100000, 
    mean = 0, 
    maxMargin = 50000, 
    volatility = 500, 
    wicks = [40, 40], 
    frequency = "5m"
)

# saveCandles(candles, location = "Candlestick-Noise/Data/Generated-1year-50,50-5min-vola-i.pkl", saveImage = True, method = "plotly")

plotDataset(candles.iloc[0:1000], method = "plotly")



100001


C:\Users\Spino.shop\AppData\Local\Temp\ipykernel_8772\3874547361.py:82: DeprecationWarning:

setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.

c:\Users\Spino.shop\Desktop\Trading\Apps\.MotherVenv\lib\site-packages\plotly\graph_objs\_deprecations.py:544: DeprecationWarning:

plotly.graph_objs.XAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.XAxis
  - plotly.graph_objs.layout.scene.XAxis


c:\Users\Spino.shop\Desktop\Trading\Apps\.MotherVenv\lib\site-packages\plotly\graph_objs\_deprecations.py:572: DeprecationWarning:

plotly.graph_objs.YAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.YAxis
  - plotly.graph_objs.layout.scene.YAxis




In [20]:

cls = GenerateCandles()

candles = cls.BrownianMotion(
    candlesCount = 100000, 
    mean = 0, 
    maxMargin = 50000, 
    volatility = 500, 
    wicks = [40, 40], 
    frequency = "5m"
)

# saveCandles(candles, location = "Candlestick-Noise/Data/Generated-1year-50,50-5min-vola-i.pkl", saveImage = True, method = "plotly")

plotDataset(candles.iloc[0:1000], method = "plotly")



c:\Users\Spino.shop\Desktop\Trading\Apps\.MotherVenv\lib\site-packages\plotly\graph_objs\_deprecations.py:544: DeprecationWarning:

plotly.graph_objs.XAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.XAxis
  - plotly.graph_objs.layout.scene.XAxis


c:\Users\Spino.shop\Desktop\Trading\Apps\.MotherVenv\lib\site-packages\plotly\graph_objs\_deprecations.py:572: DeprecationWarning:

plotly.graph_objs.YAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.YAxis
  - plotly.graph_objs.layout.scene.YAxis




In [9]:
import numpy as np
V_Volatility = np.random.uniform(low = -30, high = 30, size = 100000) # The volatility of the noise
print(len(V_Volatility))
volatility = []
for num in V_Volatility:
    volatility.append(np.random.normal(scale = 30, loc = num, size = 1))

volatility

100000


[array([-2.92957869]),
 array([-19.26539565]),
 array([7.10151009]),
 array([19.57527786]),
 array([24.82045211]),
 array([-19.37543571]),
 array([-77.16308416]),
 array([58.07141299]),
 array([-20.86977626]),
 array([20.37992324]),
 array([41.90254768]),
 array([-5.54110717]),
 array([20.18008516]),
 array([58.12770376]),
 array([-7.05227938]),
 array([-26.00568294]),
 array([0.45232688]),
 array([4.25139866]),
 array([6.70537927]),
 array([-0.11228313]),
 array([60.62615922]),
 array([-33.97596599]),
 array([35.75750283]),
 array([-35.25675906]),
 array([37.50201893]),
 array([34.36841761]),
 array([-22.75679499]),
 array([-32.56883416]),
 array([58.57379232]),
 array([49.62911881]),
 array([-18.96882748]),
 array([1.73222268]),
 array([-49.18797371]),
 array([44.16005812]),
 array([38.3478991]),
 array([-7.44617348]),
 array([-30.09785632]),
 array([-30.99870959]),
 array([-30.51115908]),
 array([-34.12010158]),
 array([14.44299615]),
 array([-27.41875721]),
 array([59.06256831]),
 